# QOSF Mentorship Program Screening Tasks

## Task 3 ZNE

Zero-noise extrapolation (ZNE) is a noise mitigation technique. It works by intentionally scaling the noise of a quantum circuit to then extrapolate the zero-noise limit of an observable of interest. In this task, you will build a simple ZNE function from scratch:

1. Build a simple noise model with depolarizing noise 
2. Create different circuits to test your noise models and choose the observable to measure 
3. Apply the unitary folding method. 
4. Apply the extrapolation method to get the zero-noise limit. Different extrapolation methods achieve different results, such as Linear, polynomial, and exponential.
5. Compare mitigated and unmitigated results 
6. Bonus: Run your ZNE function in real quantum hardware through the [IBM Quantum Service](https://www.ibm.com/quantum)

Check the [Mitiq documentation](https://mitiq.readthedocs.io/en/stable/guide/zne-5-theory.html) for references. You are not allowed to use the functions from Mitiq or any other frameworks where ZNE is already implemented. 


In [1]:
from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_aer import AerSimulator
from qiskit_aer.noise import (
    NoiseModel,
    QuantumError,
    ReadoutError,
    depolarizing_error,
    pauli_error,
    thermal_relaxation_error,
)
from qiskit.visualization import plot_histogram

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

In [26]:
# Step 1: Build a simple noise model with depolarizing noise
def create_noise_model(depolarizing_error_prob):
    noise_model = NoiseModel()
    # Depolarizing error for single-qubit gates
    depolarizing_error_1q = depolarizing_error(depolarizing_error_prob, 1)
    # Depolarizing error for two-qubit gates (cx)
    depolarizing_error_2q = depolarizing_error(depolarizing_error_prob * 2, 2)
    
    noise_model.add_all_qubit_quantum_error(depolarizing_error_1q, ['u1', 'u2', 'u3'])
    noise_model.add_all_qubit_quantum_error(depolarizing_error_2q, ['cx'])
    
    return noise_model

# Step 2: Create a simple quantum circuit to test the noise model
def create_bell_state_circuit():
    qc = QuantumCircuit(2, 2)
    qc.h(0)
    qc.cx(0, 1)
    # qc.measure([0, 1], [0, 1])
    return qc

# Step 3: Apply the unitary folding method - here we'll define a function to fold a circuit
def fold_circuit(circuit, fold_factor):
    folded_circuit = circuit.copy()
    for _ in range(fold_factor):
        for gate in reversed(circuit.data):
            folded_circuit.data.append(gate)
        for gate in circuit.data:
            folded_circuit.data.append(gate)
    return folded_circuit

def execute_circuits_with_noise(circuit, backend, noise_model, fold_factors):
    results = []
    for fold_factor in fold_factors:
        # Apply folding
        folded_circuit = fold_circuit(circuit, fold_factor)
        folded_circuit.measure([0, 1], [0, 1])
        # Execute the circuit on the backend with the noise model
        new_circuit = transpile(folded_circuit, backend)
        job = backend.run(new_circuit,noise_model=noise_model)
        # job = execute(folded_circuit, backend, noise_model=noise_model)
        result = job.result().get_counts(folded_circuit)
        results.append(result)
    return results

def linear_extrapolation(x, y):
    return np.polyfit(x, y, 1)

def exponential_extrapolation(x, y):
    fit_func = lambda x, a, b: a * np.exp(b * x)
    params, _ = curve_fit(fit_func, x, y)
    return params


In [27]:
# Test the noise model and circuit creation
noise_model = create_noise_model(0.01)  # Depolarizing error probability
qc = create_bell_state_circuit()

2

In [21]:
print("Original Quantum Circuit:")
print(qc)

Original Quantum Circuit:
     ┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘
c: 2/══════════
               


In [23]:
folded_qc = fold_circuit(qc, 1)
print("\nFolded Quantum Circuit (Factor 1):")
print(folded_qc)


Folded Quantum Circuit (Factor 1):
     ┌───┐          ┌───┐┌───┐     
q_0: ┤ H ├──■────■──┤ H ├┤ H ├──■──
     └───┘┌─┴─┐┌─┴─┐└───┘└───┘┌─┴─┐
q_1: ─────┤ X ├┤ X ├──────────┤ X ├
          └───┘└───┘          └───┘
c: 2/══════════════════════════════
                                   


In [31]:
service = QiskitRuntimeService(
    channel='ibm_quantum',
    instance='ibm-q-skku/kyunghee-univers/kh-graduate',
    token='a31caad39519f6f302e0c199c269fdf896dfd354849c28e42091c464a09318abb156bb93cb5e1f73ef095093072d01ade10d5136a7aeebf5c00a3d769a1fccac'
)
backend = service.backend("ibmq_qasm_simulator")
# Example usage with linear extrapolation
fold_factors = [0, 1, 2, 3, 4]  # Original and 3 levels of folding
results = execute_circuits_with_noise(qc, backend, noise_model, fold_factors)
# Assume 'results' is processed to extract expectation values as 'expectation_vals'
# x_vals would be your fold_factors, and y_vals would be your 'expectation_vals'
# linear_params = linear_extrapolation(x_vals, y_vals)

In [32]:
# Step 1: Extract the probabilities of the '00' outcome
probabilities = [counts['00'] / sum(counts.values()) for counts in results]

# Step 2: Perform Linear Extrapolation
# Here we use numpy's polyfit to fit a linear model (degree = 1) to the data
linear_params = linear_extrapolation(fold_factors, probabilities)

# The linear_params contains two values: [slope, intercept]
slope, intercept = linear_params

# Extrapolate to zero noise
zero_noise_prob = intercept  # Since at zero noise, the fold factor is 0, and thus y = intercept

print(f"Linear Extrapolation Parameters: slope = {slope}, intercept = {intercept}")
print(f"Extrapolated Zero-Noise Probability for '00' outcome: {zero_noise_prob}")

Linear Extrapolation Parameters: slope = 0.0015000000000000068, intercept = 0.4999
Extrapolated Zero-Noise Probability for '00' outcome: 0.4999
